Bioinformatic's exercise done by Laleh Varghaei

Convert the txt file to a CSV file.

In [1]:
import csv

input_file = '/Users/laleh/Desktop/Bioinformatics_GU/samples.txt'
output_file = '/Users/laleh/Desktop/Bioinformatics_GU/samples.csv'

# Convert the txt file to a csv file
with open(input_file, 'r') as file:
    lines = file.readlines()

with open(output_file, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    
    header = lines[0].strip().split(',')
    csv_writer.writerow(header)
    
    for line in lines[1:]:
        row = line.strip().split(',')
        csv_writer.writerow(row)


Read the CSV file.

In [2]:
import pandas as pd

data = pd.read_csv(output_file)
print(data)

       sample  pct_N_bases  pct_covered_bases  longest_no_N_run  \
0    DN-64554         3.91              96.07              7055   
1    DC-31756         4.14              95.93              7055   
2    DD-28879         4.32              95.68              9033   
3    DD-22466         3.63              96.37              7055   
4    DC-95171         4.13              95.86              8855   
..        ...          ...                ...               ...   
368  DD-23750         8.76              91.29              6683   
369  DN-86931        38.16              61.85              1595   
370  DT-97623        10.15              89.85              6489   
371  DN-11305         5.44              94.56              7057   
372  DD-41981         9.75              90.23              5793   

     num_aligned_reads  qc_pass  
0               489499     True  
1               527966     True  
2               444775     True  
3               621979     True  
4               510658   

Write a script that can be run automatically each time such a quality file is generated, and look
at how many samples from each origin fail the set quality cut-off (< 95 % covered bases of the
reference genome or ‘FALSE’ in column 6).

As this script should run automatically once a week, the script should also serve as a warning system,
sending warnings if there are certain origins producing more than 10% failed samples. Therefore, you
need to implement a system that notifies its user in some way, telling them the latest results.

In [3]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Define a function to send a warning
def send_email(subject, body, to_email):
    from_email = "laleh.v90@gmail.com"
     # I deleted the app-password for the security, an app password was created and an email successfuly was sent.
    from_password = "app-password"

    # Setup the MIME
    message = MIMEMultipart()
    message["From"] = from_email
    message["To"] = to_email
    message["Subject"] = subject

    # Add body to email
    message.attach(MIMEText(body, "plain"))

    # Create SMTP session
    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.starttls()
        server.login(from_email, from_password)
        text = message.as_string()
        server.sendmail(from_email, to_email, text)


# Identify samples that fail the threshold
failing_samples = data[(data['pct_covered_bases'] < 95.0) | (data['qc_pass'] == False)].copy()

# Extract the origin from the samples
failing_samples.loc[:, 'origin'] = failing_samples['sample'].apply(lambda x: x[1])
failing_counts = failing_samples['origin'].value_counts()
total_counts = data['sample'].apply(lambda x: x[1]).value_counts()

# The percentage of failing samples
failing_percentage = (failing_counts / total_counts) * 100

print("Number of failing samples from each origin:")
print(failing_counts)
print("\nPercentage of failing samples from each origin:")
print(failing_percentage)

# Check if any origin has more than 10% failing samples and send an email
for origin, percentage in failing_percentage.items():
    if percentage > 10:
        subject = f"Warning: High Failure Rate for Origin {origin}"
        body = f"The origin '{origin}' has a failure rate of {percentage:.2f}%.\n\n"
        body += "Number of failing samples from each origin:\n"
        body += failing_counts.to_string() + "\n\n"
        body += "Percentage of failing samples from each origin:\n"
        body += failing_percentage.to_string()
        
        send_email(subject, body, "varghaei@student.chalmers.se")


Number of failing samples from each origin:
D    32
N    27
C    20
T    12
Name: origin, dtype: int64

Percentage of failing samples from each origin:
C    25.000000
D    29.906542
N    28.125000
T    13.333333
dtype: float64
